In [1]:
import requests as req
from bs4 import BeautifulSoup as bsp
import pandas as pd
import csv
import json
import time
from selenium import webdriver

In [2]:
def getPage(url):  # function to get the html of the scrapped page for readability
    response = req.get(url)
    if not response.ok:
        print("Server responded with exit code:", response.status_code)  # if scrapping is not allowed
        return None
    else:
        soup = bsp(response.text, "lxml")  # converting to html
        pretty = soup.prettify()  # increasing readability
        with open('scrapped.html', mode='w',
                  encoding='utf-8') as htmlFile:  # specify encoding method for writing in a file
            htmlFile.write(pretty)
        return soup

In [3]:
def findContent(soup, info,last_height):  # function to extract the required elements from the website soup
    if soup is None:  # if previous function returned none
        return

    # going from basic to specific
    ol = soup.find('ul', id='product-grid')

    articles = []
    for i in range(1, last_height):
        articles += ol.find_all('li', class_=f'coll-main-wrapper grid-layout grid__item page-{i}')

    print(len(articles))
    for i in range(len(articles)):
        # extracting the title
        Sup = articles[i].find('div', class_='card-wrapper underline-links-hover')
            # extracting the Name
        try:
            Name = articles[i].find('h3', class_='card__heading')
            price = Name.text.strip()  # if price_text.startswith('£') else 0.0
            print(price)
            info['Name'].append(price)
        except:
            info['Name'].append(' ')

            # extracting the price
        try:
            Name = articles[i].find('span', class_='money')
            price = Name.text.strip()  # if price_text.startswith('£') else 0.0
            print(price)
            info['Price'].append(price)
        except:
            info['Price'].append(' ')

            # extracting the Image
        try:
            head = articles[i].find('img')
            info['Image'].append(head['src'])
        except:
            info['Image'].append(' ')   

In [4]:
 info = {  # uninitialized dictionary to use for later
        'Name': [],
        'Price': [],
        'Image': [],
    }

In [5]:
url = 'https://outfitters.com.pk/collections/men-mid-summer-sale'

In [6]:
# Using Selenium to scroll down the page
driver = webdriver.Chrome()
driver.get(url)

# code to scroll down the page to the end
SCROLL_PAUSE_TIME = 2.5
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    SCROLL_PAUSE_TIME += 0.5
    new_height = driver.execute_script("return document.body.scrollHeight")

        # If reached the end of the page
    if new_height == last_height:
            # Scroll back to top
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(SCROLL_PAUSE_TIME)
        # Scroll down to bottom again
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        break

    last_height = new_height

soup = bsp(driver.page_source, 'html.parser')
driver.quit()

In [7]:
findContent(soup, info,last_height)  # finding and extracting required stuff

741
Slim Fit Cargo Pants
PKR 4,190
Slim Fit Cargo Pants
PKR 4,190
Slim Fit Cargo Pants
PKR 4,190
Multi-Piece Sneakers
PKR 6,590
Multi-Piece Sneakers
PKR 6,590
Palm Tree Pack of 3 Socks
PKR 590
Slogan Print Shorts
PKR 2,590
Slogan Print Shorts
PKR 2,590
Basic Medium Weight T-Shirt
PKR 790
Rigid Slim Non Denim Pants
PKR 2,690
Rigid Slim Non Denim Pants
PKR 2,690
PERFORATED SNEAKERS
PKR 6,590
Basic V-Neck T-Shirt
PKR 890
Basic V-Neck T-Shirt
PKR 890
Fuax Leather Sneakers
PKR 5,890
Skinny Fit Jeans
PKR 2,190
Basic Pop Over Shirt
PKR 3,490
Basic Pop Over Shirt
PKR 3,490
Basic Pop Over Shirt
PKR 3,490
Gum Sole Contrast Sneakers
PKR 6,590
Gum Sole Contrast Sneakers
PKR 6,590
Basic V-Neck T-Shirt
PKR 890
Color Block Sneakers
PKR 6,590
Textured Flip Flops
PKR 1,790
Basic 5 Pocket Casual Trousers
PKR 3,790
Basic 5 Pocket Casual Trousers
PKR 3,790
Basic 5 Pocket Casual Trousers
PKR 3,790
Basic 5 Pocket Casual Trousers
PKR 3,790
Two-Tone Low Top Sneakers
PKR 6,290
Raglan Graphic T-shirt
PKR 990
Fu

In [8]:
# Write dictionary to CSV file
csvFile = 'Outfitters.csv'  # writing the headings of the columns
with open(csvFile, mode='w', newline='', encoding='utf-8') as file:  # using an encoder for special characters
    fieldnames = list(info.keys())  # typecasting the keys into a list
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

    for i in range(len(info['Name'])):  # according to number of items, we are copying every value of every key into it's respective heading
        row = {}
        for key in info.keys():
            row[key] = info[key][i]

        writer.writerow(row)